# <font style="color:rgb(50, 120, 229);"> Fine Tuning en Keras </font>

En el cuaderno anterior, demostramos que podíamos usar **Transfer Learning** para crear un nuevo clasificador para el conjunto de datos ASL. 

Sin embargo, mostramos que congelar toda la base convolucional con pesos preentrenados del entrenamiento de ImageNet no resultó en un clasificador altamente efectivo. Esto se debe al hecho de que los pesos de ImageNet no capturan algunas de las características únicas que se requieren para el conjunto de datos ASL. 

En este cuaderno, vamos a introducir un enfoque híbrido donde utilizamos pesos preentrenados para las primeras capas de la red (que han aprendido características más generales de ImageNet) y luego permitimos que el modelo ajuste los pesos para los bloques convolucionales subsecuentes (más las capas completamente conectadas). 

Este enfoque se llama **Fine-Tuning** porque realiza pequeños ajustes a las representaciones más abstractas del modelo reutilizado para hacerlas más relevantes para el problema en cuestión.


<center>
<img src="./images/cnn_vgg_pretrained_small_base_ASL.webp" width="800px"/>
</center>

**El proceso de Transfer Learning y Fine Tuning se resume a continuación:**

1. Instanciar la base convolucional VGG-16 con pesos preentrenados de ImageNet.
2. Configurar la base convolucional como "entrenable".
3. Congelar todas las capas en la base convolucional (EXCEPTO las últimas cuatro).
4. Agregar una capa clasificador densa para el conjunto de datos ASL.
5. Entrenar el modelo (las últimas cuatro capas de la base convolucional más el clasificador denso).


## <font style="color:rgb(50, 120, 229);"> 1. Configuración inicial </font>

In [ ]:
BATCH_SIZE = 32
LEARNING_RATE = 0.0001
EPOCHS = 50
IMG_WIDTH = 224
IMG_HEIGHT = 224

## <font style="color:rgb(50, 120, 229);"> 2. Descargar el conjunto de datos ASL </font>

In [ ]:
from google.colab import files

uploaded = files.upload()

In [ ]:
!mkdir -p /root/.kaggle
!mv kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d grassknoted/asl-alphabet

Dataset URL: https://www.kaggle.com/datasets/grassknoted/asl-alphabet
License(s): GPL-2.0




  0%|          | 0.00/1.03G [00:00<?, ?B/s]
  0%|          | 1.00M/1.03G [00:00<03:32, 5.17MB/s]
  0%|          | 4.00M/1.03G [00:00<01:13, 14.9MB/s]
  1%|          | 7.00M/1.03G [00:00<00:59, 18.3MB/s]
  1%|          | 10.0M/1.03G [00:00<00:53, 20.5MB/s]
  1%|          | 13.0M/1.03G [00:00<00:49, 21.9MB/s]
  2%|▏         | 16.0M/1.03G [00:00<00:47, 22.7MB/s]
  2%|▏         | 19.0M/1.03G [00:00<00:46, 23.5MB/s]
  2%|▏         | 22.0M/1.03G [00:01<00:44, 24.4MB/s]
  2%|▏         | 25.0M/1.03G [00:01<00:42, 25.3MB/s]
  3%|▎         | 28.0M/1.03G [00:01<00:41, 25.6MB/s]
  3%|▎         | 31.0M/1.03G [00:01<00:40, 26.3MB/s]
  3%|▎         | 34.0M/1.03G [00:01<00:39, 27.1MB/s]
  4%|▎         | 37.0M/1.03G [00:01<00:39, 27.1MB/s]
  4%|▍         | 40.0M/1.03G [00:01<00:38, 27.6MB/s]
  4%|▍         | 43.0M/1.03G [00:01<00:37, 28.0MB/s]
  4%|▍         | 46.0M/1.03G [00:01<00:36, 28.8MB/s]
  5%|▍         | 50.0M/1.03G [00:02<00:34, 30.1MB/s]
  5%|▌         | 54.0M/1.03G [00:02<00:33, 30.9MB/s]
 

In [ ]:
!unzip -q asl-alphabet.zip

## <font style="color:rgb(50, 120, 229);"> 3. Cargar el conjunto de datos ASL </font>

El conjunto de datos ASL tiene la siguiente estructura de directorios:

```bash
dataset_ASL_150/
    |______ A/
    |______ B/
    |______ C/
    |______ D/
    |______ E/
    |______ F/
    |______ G/
    |______ H/
    |______ I/
    |______ J/
    |______ K/
    |______ L/
    |______ M/
    |______ N/
    |______ O/
    |______ P/
    |______ Q/
    |______ R/
    |______ S/
    |______ T/
    |______ U/
    |______ V/
    |______ W/
    |______ X/
    |______ Y/
    |______ Z/
    |______ del/
    |______ nothing/
    |______ space/

In [ ]:
from keras.utils import image_dataset_from_directory

train_dataset = image_dataset_from_directory(
    "./dataset_ASL_150/",
    labels="inferred",
    label_mode="categorical",
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    validation_split=0.2,
    subset="training",
    shuffle=True,
    seed=42
)

In [ ]:
validation_dataset = image_dataset_from_directory(
    "./dataset_ASL_150/",
    labels="inferred",
    label_mode="categorical",
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    validation_split=0.2,
    subset="validation",
    shuffle=True,
    seed=42
)

In [ ]:
from matplotlib import pyplot as plt
import numpy as np

class_names = train_dataset.class_names

plt.figure(figsize=(10, 10))

for images, labels in train_dataset.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        label = np.argmax(labels[i])
        class_name = class_names[label]
        plt.title(class_name)
        plt.axis("off")
    break
plt.show()

## <font style="color:rgb(50, 120, 229);"> 4. Crea el modelo </font>

<font style="color:rgb(50, 120, 229);"> ¿Cómo aplicar Fine Tuning en Keras? </font>

El enfoque general para el **Fine Tuning** de una base convolucional es primero cargar los pesos preentrenados para el modelo y luego permitir selectivamente que las últimas capas convolucionales de la base sean **entrenables**. 

El proceso de especificar selectivamente qué capas son entrenables y cuáles no a menudo se denomina congelar o descongelar capas mediante el atributo **`trainable`** del modelo. 

En la figura siguiente, mostramos que la base convolucional se ha cargado con pesos preentrenados de ImageNet. 


Luego "congelamos" la primera parte de la base convolucional para preservar las características aprendidas de ImageNet, pero permitimos que las últimas cuatro capas se ajusten durante el proceso de entrenamiento. 

Usamos el término "Fine Tuning" en este contexto porque esperamos que las últimas capas de la base convolucional (preentrenada en ImageNet) contengan representaciones de características que son "bastante buenas", pero que requieren una mayor afinación para ser más relevantes para el conjunto de datos ASL.

<center>
<img src="./images/cnn_vgg_pretrained_fine_tune.png" width="600px"/>
</center>

<font style="color:rgb(50, 120, 229);"> ¿Cuál es la forma correcta de hacer Fine Tuning? </font>

Idealmente, deberías entrenar primero las capas densas (manteniendo toda la base convolucional congelada) como hicimos en el caso de "Transfer Learning" y luego comenzar a ajustar las capas convolucionales a una tasa de aprendizaje más baja.

Sin embargo, para mantener las cosas simples en este ejemplo, entrenaremos el modelo solo una vez, lo que incluirá las últimas capas de la base convolucional así como el clasificador denso.

En la práctica, al ajustar un modelo, es mejor comenzar con Fine Tuning de solo unas pocas capas a la vez para ver cómo responde el modelo.

### <font style="color:rgb(50, 120, 229);"> 4.1 Cargar la base convolucional VGG16 con pesos preentrenados </font>

Comenzamos creando un modelo de la base convolucional VGG-16. Podemos hacer esto instanciando el modelo y configurando `include_top = False`, lo que excluye las capas completamente conectadas.

In [ ]:
from keras.applications import VGG16
from keras.models import Model 
from keras.layers import Dense, Flatten, Input

base_model = VGG16(
    weights="imagenet",
    include_top=False,
    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)
)

base_model.trainable = True

### <font style="color:rgb(50, 120, 229);"> 4.2 Descongelar las últimas capas de la base convolucional </font>

Configuramos el atributo `trainable` de la base convolucional a `True`. Esto ahora nos permite "congelar" un número seleccionado de capas en la base convolucional para que solo las últimas capas de la base convolucional sean entrenables.


In [ ]:
num_layers_fine_tune = 4
num_layers = len(base_model.layers)

for layer in base_model.layers[:num_layers - num_layers_fine_tune]:
    print(f"Layer {layer.name} will be frozen")
    layer.trainable = False

### <font style="color:rgb(50, 120, 229);"> 4.3 Agregar un clasificador</font>

Dado que pretendemos entrenar y usar el modelo para clasificar señales de mano del conjunto de datos ASL (que tiene 29 clases), necesitaremos agregar nuestra propia capa de clasificación. 

En este ejemplo, hemos elegido usar solo una capa densa completamente conectada que contiene 256 nodos, seguida de una capa de salida softmax que contiene 29 nodos, uno para cada una de las 29 clases. 

El número de capas densas y el número de nodos por capa es una elección de diseño, pero el número de nodos en la capa de salida debe coincidir con el número de clases en el conjunto de datos.


In [ ]:
from keras.applications.vgg16 import preprocess_input

input_layer = Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))

x = preprocess_input(input_layer)
x = base_model(x)
x = Flatten()(x)
x = Dense(256, activation="relu")(x)

output_layer = Dense(29, activation="softmax")(x)

model = Model(input_layer, output_layer)
model.summary()

### <font style="color:rgb(50, 120, 229);"> 4.4 Compilar y entrenar el modelo </font>

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam

model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

early_stopping = EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True
)

model_checkpoint = ModelCheckpoint(
    "best_model.keras",
    monitor="val_loss",
    save_best_only=True
)

In [ ]:
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=EPOCHS,
    callbacks=[early_stopping, model_checkpoint]
)

## <font style="color:rgb(50, 120, 229);"> 5. Gráficos de pérdida y precisión </font>

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history["accuracy"], label="accuracy")
plt.plot(history.history["val_accuracy"], label="val_accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.show()

plt.plot(history.history["loss"], label="loss")
plt.plot(history.history["val_loss"], label="val_loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.show()


## <font style="color:rgb(50, 120, 229);"> 6. Predicciones </font>

In [ ]:
from keras.models import load_model

model = load_model("best_model.keras")

class_names = train_dataset.class_names

for images, labels in validation_dataset.take(1):
    predictions = model.predict(images)
    plt.figure(figsize=(20, 10))
    
    for i in range(24):
        image = images[i]
        prediction = predictions[i].argmax()
        label = labels[i].numpy().argmax()

        pred_class_name = class_names[prediction]
        label_class_name = class_names[label]
        plt.subplot(4, 6, i + 1)
        plt.imshow(image.numpy().astype("uint8"))
        plt.title(f"Prediction: {pred_class_name}, Label: {label_class_name}")
        plt.axis("off")

    break

plt.show()
        

Vamos a mostrar la matriz de confusión para ver cómo se desempeña el modelo en el conjunto de datos de validación.

In [ ]:
from keras.metrics import TrueNegatives, TruePositives, FalseNegatives, FalsePositives

# Create a confusion matrix
tn = TrueNegatives()
tp = TruePositives()
fn = FalseNegatives()
fp = FalsePositives()


for images, labels in validation_dataset:
    predictions = model.predict(images)
    tn.update_state(labels, predictions)
    tp.update_state(labels, predictions)
    fn.update_state(labels, predictions)
    fp.update_state(labels, predictions)

print("True Negatives: ", tn.result().numpy())
print("True Positives: ", tp.result().numpy())
print("False Negatives: ", fn.result().numpy())
print("False Positives: ", fp.result().numpy())

In [ ]:
import seaborn as sns

confusion_matrix = [
    [tn.result().numpy(), fp.result().numpy()],
    [fn.result().numpy(), tp.result().numpy()]
]

sns.heatmap(confusion_matrix, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted")